In [1]:
import os
import pandas as pd
import numpy as np

# Default packages for the minimum example
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, f1_score


import pickle #for saving/loading trained classifiers


In [18]:
#Where are the files
file_data = '.' + os.sep + 'data' + os.sep +'metadata.csv'
df = pd.read_csv(file_data)
label = np.array(df['diagnostic'])

path_image = '.' + os.sep + 'data' + os.sep + 'images' + os.sep + 'imgs_part_1'

#Where did we store the features?
file_features = 'features/features.csv'
feature_names = ['file_name','asymmetry','color','blue-white_veil']

# Load the features - remember the example features are not informative
df_features = pd.read_csv(file_features)


our_list = list(np.array(df_features["file_name"]))
filtered_data = df[df["img_id"].isin(our_list)]
filtered_label = np.array(filtered_data['diagnostic'])
image_id = list(filtered_data['img_id'])
filtered_data

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,PAT_1761,3329,NaN,NaN,NaN,NaN,70,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1761_3329_837.png,False
1153,PAT_564,1079,True,False,BRAZIL,BRAZIL,73,False,FEMALE,False,...,8.0,BCC,True,True,False,True,True,True,PAT_564_1079_430.png,True
1154,PAT_747,1409,False,False,UNK,BRAZIL,55,False,MALE,True,...,10.0,ACK,True,True,False,False,True,True,PAT_747_1409_116.png,True
1155,PAT_566,179,False,False,ITALY,BRAZIL,54,False,MALE,False,...,3.0,ACK,True,False,False,False,False,False,PAT_566_179_23.png,False


In [70]:
# Make the dataset, you can select different classes (see task 0)
x = np.array(df_features[feature_names[1:]])
y =  np.int_(filtered_label == 'MEL')   #now True means healthy nevus, False means something else
patient_id = filtered_data['patient_id']


In [56]:
patient_id

976      PAT_330
983      PAT_373
984     PAT_1221
988      PAT_150
989      PAT_192
          ...   
1185     PAT_901
1188     PAT_467
1342     PAT_270
1348     PAT_801
1424     PAT_622
Name: patient_id, Length: 124, dtype: object

In [57]:
#Prepare cross-validation - images from the same patient must always stay together
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)
group_kfold.get_n_splits(x, y, patient_id)


5

In [58]:
group_kfold

GroupKFold(n_splits=5)

In [59]:
#Different classifiers to test out
classifiers = [
    KNeighborsClassifier(1),
    KNeighborsClassifier(5)
]
num_classifiers = len(classifiers)     

In [85]:
acc_val = np.empty([num_folds,num_classifiers])
f1_val = np.empty([num_folds, num_classifiers])

In [109]:
for i, (train_index, val_index) in enumerate(group_kfold.split(x, y, patient_id)):
    
    x_train = x[train_index,:]
    y_train = y[train_index]
    x_val = x[val_index,:]
    y_val = y[val_index]
    
    
    for j, clf in enumerate(classifiers): 
        
        #Train the classifier
        clf.fit(x_train,y_train)
    
        #Evaluate your metric of choice (accuracy is probably not the best choice)
        acc_val[i,j] = accuracy_score(y_val, clf.predict(x_val))
        f1_val[i,j] = f1_score(y_val, clf.predict(x_val), average="binary")


C:\Users\mhm25\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Users\mhm25\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [75]:
#Average over all folds
average_acc = np.mean(acc_val,axis=0) 
   
print('Classifier 1 average accuracy={:.3f} '.format(average_acc[0]))
print('Classifier 2 average accuracy={:.3f} '.format(average_acc[1]))



Classifier 1 average accuracy=0.960 
Classifier 2 average accuracy=0.976 


In [76]:
#Let's say you now decided to use the 5-NN 
classifier = KNeighborsClassifier(n_neighbors = 5)

#It will be tested on external data, so we can try to maximize the use of our available data by training on 
#ALL of x and y
classifier = classifier.fit(x,y)

#This is the classifier you need to save using pickle, add this to your zip file submission
filename = 'groupXY_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))